In [ ]:
#BSM模型定价
from math import log, e, sqrt, pi
from scipy import stats #用于统计分析，里面有stats.norm.cdf是正态分布的累计概率密度方程
import pandas as pd

def get_option_price(S, X, r, q, sigma, T, opt_type='call'): #BSM模型定价
    """
    根据BSM模型计算期权价格
    param S: 标的资产当前价格
    param X: 期权行权价格
    param r: 无风险收益率
    param q: 基础资产分红收益率
    param sigma：标的资产年化波动率
    param T: 以年计算的期权到期时间
    param opt_type: 'call'或'put'
    return 返回期权价格
    """
    d1 = (log(S/X) + (r - q + sigma**2/2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    N1, N2 = stats.norm.cdf([d1, d2])
    call = S *e ** (-q * T) N1 - X * e ** (-r * T) * N2 #计算认购期权价格

    if opt_type.lower()[0] == 'c':
        return call
    else: 
        put = call + X * e ** (-r * T) - S  # 利用期权平价模型计算put价格
        return put

get_option_price(2.47, 2.25, 0.05, 0, 0.325, 24/365, 'call') #期权到期时间如果是工作日则T=到期时间/252，如果包括了非工作日则T=到期时间/365
get_option_price(2.47, 2.25, 0.05, 0, 0.325, 24/365, 'put')

#计算历史波动率
import tushare as ts
import pandas as pd
import pandas_datareader.data as web

etf_return = ts.get_k_data('510050', '2017-11-01').set_index('date').close.pct_change().dropna() #tushare获取标的资产日收益率

#也可以用DataReader获取标的资产日收益率
start = datetime(2017, 1, 1)
# end = datetime(2017, 5, 24)
aapl = web.DataReader('AAPL', 'iex', start)

sigma = etf_return.rolling(60).std()[-1] * sqrt(252) # 根据标的资产日收益率计算60个交易日移动年化历史波动率

#使用BSM模型计算不同行权价格的期权价格
S = 2.4
sigma = 0.27444
bottom_strike = 2.2
r = 0.0246
q = 0
T = 15/365
opt_price = []

for i in range(16):
    strike = bottom_strike + i * 0.05
    call = get_option_price(S, strike, r, q, sigma, T)
    put = get_option_price(S, strike, r, q, sigma, T, opt_type='put')
    opt_price.append([strike, call, put])  
df = pd.DataFrame(opt_price, columns=['strike', 'call', 'put'])

#牛顿法计算隐含波动率

def vega(S, X, r, q, sigma, T): #计算Vega(期权价格对隐含波动率的一阶导)
    """
    定义函数计算期权的Vega
    param S: 标的资产当前价格
    param X: 期权行权价格
    param r: 复合无风险收益率
    param q: 基础资产分红收益率
    param sigma：标的资产年化波动率
    param T: 以年计算的期权到期时间
    return：返回期权的 Vega 值
    """
    d1 = (log(e**(-q*T)*S/X) + (r + sigma**2/2) * T) / (sigma * sqrt(T))
    N_dash = e ** (-d1 ** 2 / 2)/sqrt(2 * pi)
    return S * N_dash * sqrt(T)

S = 2.4
X = 2.45
r = 0.0246
q = 0
sigma = 0.27
T = 16/365

vega(S, X, r, q, sigma, T)

def get_implied_vol(S, X, r, q, T, V_mkt, opt_type='call'): #牛顿法计算隐含波动率，即求g(σ)=f(σ)−期权实际价格=0 ，g'(σ)=f'(σ)=Vega
    """
    根据期权数据计算隐含波动率
    param S: 标的资产当前价格
    param X: 期权行权价格
    param r: 复合无风险收益率
    param q: 基础资产分红收益率
    param T: 以年计算的期权到期时间
    param V_mkt: 期权的市场价格
    param opt_type：期权类型，put或call
    return：返回隐含波动率
    """
    sigma = 0.5 #初始化用于逐步逼近的sigma
    error = 10**(-6) #精度，如果精度满足则可以提前结束循环不需要迭代100次
    
    for i in range(100):
        V = get_option_price(S, X, r, q, sigma, T, opt_type) # BSM计算的期权价格
        if abs(V - V_mkt) < error: #如果计算出的期权价格和期权市场价格之差小于error
            return sigma
        sigma = sigma - (V - V_mkt)/vega(S, X, r, q, sigma, T) #迭代计算sigma，g(x)=(V-V_mkt)，g'(x)=vega，vega(sigma1-sigma0)=y-g(sigma0)，y=0求解
        print(i, sigma)
    return sigma

S = 2.41
X = 2.4
r = 0.024839
q = 0
T = 43/365
V_mkt = 0.0821
opt_type = 'c'

get_implied_vol(S, X, r, q, T, V_mkt, opt_type)